In [8]:
import logging
logging.basicConfig(level=logging.INFO)

## __get__(self, instance, owner)

instance 는 디스크립터를 호출한 인스턴스를 의미한다. (아래 예제에서 client 인스턴스)

owner 는 디스크립터를 호출한 인스턴스의 Class를 의미한다. (아래 예제에서 ClientClass 의미)

In [9]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return f"{self.__class__.__name__}.{owner.__name__}"
        return f"value for {instance}"
    
class ClientClass:
    descriptor = DescriptorClass()

ClientClass.descriptor  #<-- Class

'DescriptorClass.ClientClass'

In [10]:
ClientClass().descriptor  #<-- Instance


'value for <__main__.ClientClass object at 0x0000020293680188>'

In [4]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        logging.info("Call: %s.__get__(%r, %r)", self.__class__.__name__, instance, owner)
        print ('1.')
        print (instance.__dict__)
        print ('2.')
        print (owner.__dict__)
        return instance

class ClientClass:
    descriptor = DescriptorClass()

In [5]:
print ('min')
client = ClientClass()
print ('--')
client.descriptor

min
--
1.
{}
2.
{'__module__': '__main__', 'descriptor': <__main__.DescriptorClass object at 0x000002029375B148>, '__dict__': <attribute '__dict__' of 'ClientClass' objects>, '__weakref__': <attribute '__weakref__' of 'ClientClass' objects>, '__doc__': None}


In [7]:
print (client.__dict__)

{}


## __set__(self, instance, value)

In [12]:
class Validation:
    def __init__(self, validation_function, error_msg:str):
        self.validation_function = validation_function
        self.error_msg = error_msg
    def __call__(self, value):
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")

class Field:
    def __init__(self, *validations):
        self._name = None
        self.validations = validations
    def __set_name__(self, owner, name):
        self._name = name
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]
    def validate(self, value):
        for validation in self.validations:
            validation(value)
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self._name] = value

class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), "는 숫자가 아님"),
        Validation(lambda x: x >=0, "는 0보다 작음"),        
    )

In [14]:
client = ClientClass()

In [15]:
client.descriptor = 42

In [16]:
client.descriptor

42

In [17]:
client.descriptor = -42

ValueError: -42 는 0보다 작음

In [18]:
client.descriptor = "invalid value"

ValueError: 'invalid value' 는 숫자가 아님